# Adding the $8\pi T^{\mu\nu}$ Matter Source Terms to Einstein's equations of general relativity in the [BSSN](http://www2.yukawa.kyoto-u.ac.jp/~yuichiro.sekiguchi/3+1.pdf) formalism, in ***curvilinear*** coordinates, using a covariant reference metric approach: C code generation of the evolution equations' right-hand sides

## Python module containing these expressions: [BSSN/BSSN_RHS.py](../edit/BSSN/BSSN_RHSs.py)

### ***Citations***: Generic curvilinear coordinate reference metric approach matches that of [Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/abs/1712.07658), which is an extension of the spherical coordinate reference metric approach of [Baumgarte, Montero, Cordero-Carrión, and Müller (2012)](https://arxiv.org/abs/1211.6632), which builds upon the covariant "Lagrangian" BSSN formalism of [Brown (2009)](https://arxiv.org/abs/0902.3652). *See also citations within each article.*

In the [previous module](Tutorial-BSSNCurvilinear.ipynb), we generated SymPy expressions for all of the covariant BSSN equations in curvilinear coordinates, rescaling tensors to factor out coordinate singularities (and render them harmless). To wit:

\begin{align}
  \partial_t \bar{\gamma}_{i j} {} = {} & \left[\beta^k \partial_k \bar{\gamma}_{ij} + \partial_i \beta^k \bar{\gamma}_{kj} + \partial_j \beta^k \bar{\gamma}_{ik} \right] + \frac{2}{3} \bar{\gamma}_{i j} \left (\alpha \bar{A}_{k}^{k} - \bar{D}_{k} \beta^{k}\right ) - 2 \alpha \bar{A}_{i j} \; , \\
  \partial_t \bar{A}_{i j} {} = {} & \left[\beta^k \partial_k \bar{A}_{ij} + \partial_i \beta^k \bar{A}_{kj} + \partial_j \beta^k \bar{A}_{ik} \right] - \frac{2}{3} \bar{A}_{i j} \bar{D}_{k} \beta^{k} - 2 \alpha \bar{A}_{i k} {\bar{A}^{k}}_{j} + \alpha \bar{A}_{i j} K \nonumber \\
  & + e^{-4 \phi} \left \{-2 \alpha \bar{D}_{i} \bar{D}_{j} \phi + 4 \alpha \bar{D}_{i} \phi \bar{D}_{j} \phi  + 4 \bar{D}_{(i} \alpha \bar{D}_{j)} \phi - \bar{D}_{i} \bar{D}_{j} \alpha + \alpha \bar{R}_{i j} \right \}^{\text{TF}} \; , \\
  \partial_t \phi {} = {} & \left[\beta^k \partial_k \phi \right] + \frac{1}{6} \left (\bar{D}_{k} \beta^{k} - \alpha K \right ) \; , \\
  \partial_{t} K {} = {} & \left[\beta^k \partial_k K \right] + \frac{1}{3} \alpha K^{2} + \alpha \bar{A}_{i j} \bar{A}^{i j} - e^{-4 \phi} \left (\bar{D}_{i} \bar{D}^{i} \alpha + 2 \bar{D}^{i} \alpha \bar{D}_{i} \phi \right ) \; , \\
  \partial_t \bar{\Lambda}^{i} {} = {} & \left[\beta^k \partial_k \bar{\Lambda}^i - \partial_k \beta^i \bar{\Lambda}^k \right] + \bar{\gamma}^{j k} \hat{D}_{j} \hat{D}_{k} \beta^{i} + \frac{2}{3} \Delta^{i} \bar{D}_{j} \beta^{j} + \frac{1}{3} \bar{D}^{i} \bar{D}_{j} \beta^{j} \nonumber \\
  & - 2 \bar{A}^{i j} \left (\partial_{j} \alpha - 6 \partial_{j} \phi \right ) + 2 \alpha \bar{A}^{j k} \Delta_{j k}^{i}  -\frac{4}{3} \alpha \bar{\gamma}^{i j} \partial_{j} K \\
\partial_t \alpha &= \left[\beta^i \partial_i \alpha\right] - 2 \alpha K \\
  \partial_{t} \beta^{i} &= \left[\beta^j \partial_j \beta^i\right] + B^{i} \\
  \partial_{t} B^{i} &= \left[\beta^j \partial_j B^i\right] + \frac{3}{4} \partial_{0} \bar{\Lambda}^{i} - \eta B^{i}, \; .
\end{align}

where the terms moved from the left-hand side are enclosed in square braces. Note that $\partial_{0} \bar{\Lambda}^{i}$ in the right-hand side of the $\partial_{t} B^{i}$ equation is computed by adding $\beta^j \partial_j \bar{\Lambda}^i$ to the right-hand side expression given for $\partial_t \bar{\Lambda}^i$, so no explicit time dependence occurs in the right-hand sides of the BSSN evolution equations and the Method of Lines can be applied directly.

Of course the above BSSN evolution equations cannot appear as written above in our actual code, because (as discussed in the [previous module](Tutorial-BSSNCurvilinear.ipynb)) tensorial expressions can diverge at coordinate singularities. So the equations above will simply need to be rescaled according to the prescription (again, described in the [previous module](Tutorial-BSSNCurvilinear.ipynb)).

## Adding the matter source terms

Analyzing Eqs. 9 of [Baumgarte, Montero, Cordero-Carrión, and Müller](https://arxiv.org/pdf/1211.6632.pdf), we see that matter source terms $T_{\mu\nu}$ to Einstein's equations only adjust the right-hand sides of the $\partial_t \bar{A}_{ij}$, $\partial_t K$, and $\partial_t \bar{\Lambda}^i$ equations, as follows:


\begin{array}
\ \partial_t \bar{A}_{ij} &= [\text{as above}]\ {\color{blue}{-\ 8\pi \alpha e^{4\phi} \left(S_{ij}\right)^{\rm TF}}} \\
\partial_t K &= [\text{as above}]\ {\color{blue}{+\ 4\pi \alpha (\rho + S)}} \\
\partial_t \bar{\Lambda}^i &= [\text{as above}]\ {\color{blue}{-\ 16\pi \alpha \gamma^{ij} S_j}}, \\
\end{array}

where $\rho$, $S$, $S_i$, and $S_{ij}$ are related to the stress-energy tensor $T^{\mu\nu}$ as follows (note that Greek indices are 4-dimensional and Latin indices are 3-dimensional):

\begin{array}
\ S_{ij} &= \gamma_{i \mu} \gamma_{j \nu} T^{\mu \nu} \\
S_{i} &= -\gamma_{i\mu} n_\nu T^{\mu\nu} \\
S &= \gamma^{ij} S_{ij} \\
\rho &= n_\mu n_\nu T^{\mu\nu},
\end{array}

where the definition of 3-metric (Eq. 2.27 in Baumgarte & Shapiro) implies

$$\gamma_{\mu \nu} = g_{\mu \nu} + n_\mu n_\nu.$$

Here, $n_\mu = (-\alpha,0,0,0)$ is the normal one-form on each spatial slice, and

$$
g_{\mu\nu} = \begin{pmatrix} 
-\alpha^2 + \beta^k \beta_k & \beta_i \\
\beta_j & \bar{\gamma}_{ij}
\end{pmatrix}.
$$

## Numerical Implementation

The above expressions are written in backwards order from what we need in our numerical implementation, as for example $S_{ij}$ depends on $\gamma_{i\mu}$, which depends on $n_{\mu}$ and $g_{\mu\nu}$; $g_{\mu\nu}$ depends on known quantities $\bar{\gamma}_{ij}$, $\alpha$, and $\beta^i$; and $\beta_k = \bar{\gamma}_{ik} \beta^i$.

So let's set about doing this in the proper order.

### Step 1: Load needed modules and set NRPy+ parameters
First we load the needed NRPy+ modules and set parameters for this module.

In [1]:
import sympy as sp
import NRPy_param_funcs as par
import indexedexp as ixp
import reference_metric as rfm
import BSSN.BSSN_RHSs as bssn
bssn.BSSN_RHSs()

thismodule = "BSSN_RHSs_matter_source_terms"
PI = par.Cparameters("BUILTIN",thismodule,["M_PI"])

# We choose dimension DIM=3 here, as our base variables use a 3+1 decomposition.
DIM = 3

### Step 2: Set the 4-metric $g_{\mu \nu}$

$$
g_{\mu\nu} = \begin{pmatrix} 
-\alpha^2 + \beta^k \beta_k & \beta_i \\
\beta_j & \bar{\gamma}_{ij}
\end{pmatrix},
$$
which are all provided by the BSSN_RHSs module, except we need $\beta_k = \bar{\gamma}_{ik} \beta^i$, which is provided by B&S (Baumgarte & Shapiro, *Numerical Relativity*) Eq. 2.121:

In [2]:
# To get \gamma_{\mu \nu} = gammabar4DD[mu][nu], we'll need to construct the 4-metric, using Eq. 2.122 in B&S:
g4DD = ixp.zerorank2(DIM=4)

# Eq. 2.121 in B&S
betaD = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        betaD[i] += bssn.gammabarDD[i][j]*bssn.betaU[j]

# Now compute the beta contraction.
beta2 = sp.sympify(0)
for i in range(DIM):
    beta2 += bssn.betaU[i]*betaD[i]

# Eq. 2.122 in B&S
g4DD[0][0] = -bssn.alpha**2 + beta2
for mu in range(1,4):
    g4DD[mu][0] = g4DD[0][mu] = betaD[mu-1]
for mu in range(1,4):
    for nu in range(1,4):
        g4DD[mu][nu] = bssn.gammabarDD[mu-1][nu-1]

### Step 3: Set $\gamma_{\mu\nu} = g_{\mu\nu} + n_\mu n_\nu$ (B&S Eq. 2.27)

Eq. 2.117 in B&S defines $n_\mu = \{-\alpha,0,0,0\}$, and we have already declared $g_{\mu\nu} = \text{g4DD[mu][nu]}$ above, so we have all we need to compute $\gamma_{\mu\nu} = g_{\mu\nu} + n_\mu n_\nu$:

In [3]:
# n_{\mu}: Eq. 2.117 in B&S
nD  = [-bssn.alpha,sp.sympify(0),sp.sympify(0),sp.sympify(0)]

# Eq. 2.27 in B&S
gammabar4DD = ixp.zerorank2(DIM=4)
for mu in range(4):
    for nu in range(4):
        gammabar4DD[mu][nu] = g4DD[mu][nu] + nD[mu]*nD[nu]

### Step 4: Compute $S_{ij}$, $S_{i}$, $S$, and $\rho$, via Eqs. 10 of [Baumgarte, Montero, Cordero-Carrión, and Müller](https://arxiv.org/pdf/1211.6632.pdf):

\begin{array}
\ S_{ij} &= \gamma_{i \mu} \gamma_{j \nu} T^{\mu \nu} \\
S_{i} &= -\gamma_{i\mu} n_\nu T^{\mu\nu} \\
S &= \gamma^{ij} S_{ij} \\
\rho &= n_\mu n_\nu T^{\mu\nu},
\end{array}

In [4]:
# T4UU[mu][nu] = T^{\mu\nu} (the *un-rescaled* stress-energy tensor) is assumed to be provided as input.
T4UU = ixp.register_gridfunctions_for_single_rank2("AUX","T4UU", "sym01",DIM=4)

SDD = ixp.zerorank2()
SD  = ixp.zerorank1()
S   = sp.sympify(0)
rho = sp.sympify(0)

# Compute S_{ij}:
for i in range(DIM):
    for j in range(DIM):
        for mu in range(4):
            for nu in range(4):
                SDD[i][j] += gammabar4DD[i][mu]*gammabar4DD[j][nu]*T4UU[mu][nu]

# Compute S_{i}
for i in range(DIM):
    for mu in range(4):
        for nu in range(4):
            SD[i] += -gammabar4DD[i][mu]*nD[nu]*T4UU[mu][nu]

# Compute S
for i in range(DIM):
    for j in range(DIM):
        S += bssn.gammabarUU[i][j]*SDD[i][j]

# Compute rho
for mu in range(4):
    for nu in range(4):
        rho += nD[mu]*nD[nu]*T4UU[mu][nu]

### Step 5: Add matter source terms to BSSN right-hand sides *a la* Eqs. 9 of [Baumgarte, Montero, Cordero-Carrión, and Müller](https://arxiv.org/pdf/1211.6632.pdf):

\begin{array}
\ \partial_t \bar{A}_{ij} &= [\text{vacuum terms}]\ {\color{blue}{-\ 8\pi \alpha e^{4\phi} \left(S_{ij}\right)^{\rm TF}}} \\
\partial_t K &= [\text{vacuum terms}]\ {\color{blue}{+\ 4\pi \alpha (\rho + S)}} \\
\partial_t \bar{\Lambda}^i &= [\text{vacuum terms}]\ {\color{blue}{-\ 16\pi \alpha \gamma^{ij} S_j}}, \\
\end{array}

which require that we first compute $S_{ij}^{\rm TF} = \gamma^{ij} S_{ij}$. 

After setting these expressions, we apply the usual rescalings:

In [5]:
# Step 1: Add matter source term to RHS of \partial_t \bar{A}_{ij} equation:
# Step 1a: Compute trace of S_{ij}:
S_trace = sp.sympify(0)
for i in range(DIM):
    for j in range(DIM):
        S_trace += bssn.gammabarUU[i][j]*SDD[i][j]
        
# Step 1b: find the trace-free S_{ij}, and add *rescaled* matter source term to Abar_RHS:
S_TFDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        S_TFDD[i][j] = SDD[i][j] - sp.Rational(1,3)*bssn.gammabarDD[i][j]*S_trace
        bssn.a_rhsDD[i][j] += -8*PI*bssn.alpha/bssn.exp_m4phi * S_TFDD[i][j] / rfm.ReDD[i][j]

# Step 2: Add matter source term to RHS of \partial_t K equation:
bssn.trK_rhs += 4*PI*bssn.alpha*(rho + S)

# Step 3: Next add the *rescaled* matter source term to Lambar_RHS.
# Step 3a: Define the matter source term
Lambar_sourcetermU = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        Lambar_sourcetermU[i] += -16*PI*bssn.alpha*bssn.gammabarUU[i][j]*SD[j]

# Step 3b: Add the *rescaled* matter source term just computed to lambda_RHSU[i]:
for i in range(DIM):
    bssn.lambda_rhsU[i] += Lambar_sourcetermU[i] / rfm.ReU[i]

In [6]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-BSSNCurvilinear_Plus_Matter_Sources.ipynb
!pdflatex -interaction=batchmode Tutorial-BSSNCurvilinear_Plus_Matter_Sources.tex
!pdflatex -interaction=batchmode Tutorial-BSSNCurvilinear_Plus_Matter_Sources.tex
!pdflatex -interaction=batchmode Tutorial-BSSNCurvilinear_Plus_Matter_Sources.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-BSSNCurvilinear_Plus_Matter_Sources.ipynb to latex
[NbConvertApp] Writing 38355 bytes to Tutorial-BSSNCurvilinear_Plus_Matter_Sources.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
